# Question 1

In [151]:
import pandas as pd
import numpy as np

dat = pd.read_excel('ConcessionSalesData_ForClass.xlsx')
dat.head(5)

,food_game,UserID,UseCount,revenue,game_week,special_discount,special_item,FAMILYGROUPNAME,Master_Item,MENUITEMNAME,...,first_week_discount,Discount_HotDog,Discount_SouvCup,Discount_BtlWater,Discount_Peanuts,Discount_Nachos,Discount_Pretzel,Discount_Popcorn,sth_rev_game,total_product_rev_nonSTH
0,BAG PEANUTS_Game 1,3304107,1,4.726207,Game 1,STH Discount Only,Yes,SNACKS,20500003,BAG PEANUTS,...,No Discount,Yes,No,Yes,No,No,No,No,16441.58141,15296.65411
1,BAG PEANUTS_Game 1,3405989,1,4.730000,Game 1,STH Discount Only,Yes,SNACKS,20500003,BAG PEANUTS,...,No Discount,Yes,No,Yes,No,No,No,No,16441.58141,15296.65411
2,BAG PEANUTS_Game 1,3302989,1,4.730000,Game 1,STH Discount Only,Yes,SNACKS,20500003,BAG PEANUTS,...,No Discount,Yes,No,Yes,No,No,No,No,16441.58141,15296.65411
3,BAG PEANUTS_Game 1,3253641,1,4.567500,Game 1,STH Discount Only,Yes,SNACKS,20500003,BAG PEANUTS,...,No Discount,Yes,No,Yes,No,No,No,No,16441.58141,15296.65411
4,BAG PEANUTS_Game 1,3315665,1,4.726615,Game 1,STH Discount Only,Yes,SNACKS,20500003,BAG PEANUTS,...,No Discount,Yes,No,Yes,No,No,No,No,16441.58141,15296.65411


## Assumptions

- Customer spending habits differ by game, so we must control for it
- Assume that the average pricepoint for an item in each game is the weighted average of all actual prices, weighted by the demand.
- Assume the occurence of discounts on 1 item does not depend on occurence of discounts of another item

## Separate Items

In [152]:
dat['MENUITEMNAME'].unique()

array(['BAG PEANUTS', 'BAVARIAN PRETZEL', 'BRATWURST', 'BTL DEJA BLUE 1L',
       'BTL DEJA BLUE', 'BURG CHS', 'BURG VEGGIE', 'CHICAGO DOG',
       'CHIX TENDER BSKT', 'COFFEE 12', 'FRIES BSKT', 'HOT DOG',
       'NACHOS SUPREME', 'NACHOS', 'POPCORN SML', 'SAND ITLIAN BEEF',
       'SAUSAGE ITALIAN', 'SAUSAGE POLISH', 'SD CHEESE CUP',
       'SLC PIZ CHEESE', 'SLC PIZ PEPPRNI', 'SLC PIZ SAUSAGE',
       'SODA CUP 16', 'SOUV CUP 32', 'SOUV POPCORN'], dtype=object)

### Peanuts

In [153]:
peanuts = dat.loc[dat['MENUITEMNAME'] == 'BAG PEANUTS', :]
peanuts.head(5)

,food_game,UserID,UseCount,revenue,game_week,special_discount,special_item,FAMILYGROUPNAME,Master_Item,MENUITEMNAME,...,first_week_discount,Discount_HotDog,Discount_SouvCup,Discount_BtlWater,Discount_Peanuts,Discount_Nachos,Discount_Pretzel,Discount_Popcorn,sth_rev_game,total_product_rev_nonSTH
0,BAG PEANUTS_Game 1,3304107,1,4.726207,Game 1,STH Discount Only,Yes,SNACKS,20500003,BAG PEANUTS,...,No Discount,Yes,No,Yes,No,No,No,No,16441.58141,15296.65411
1,BAG PEANUTS_Game 1,3405989,1,4.730000,Game 1,STH Discount Only,Yes,SNACKS,20500003,BAG PEANUTS,...,No Discount,Yes,No,Yes,No,No,No,No,16441.58141,15296.65411
2,BAG PEANUTS_Game 1,3302989,1,4.730000,Game 1,STH Discount Only,Yes,SNACKS,20500003,BAG PEANUTS,...,No Discount,Yes,No,Yes,No,No,No,No,16441.58141,15296.65411
3,BAG PEANUTS_Game 1,3253641,1,4.567500,Game 1,STH Discount Only,Yes,SNACKS,20500003,BAG PEANUTS,...,No Discount,Yes,No,Yes,No,No,No,No,16441.58141,15296.65411
4,BAG PEANUTS_Game 1,3315665,1,4.726615,Game 1,STH Discount Only,Yes,SNACKS,20500003,BAG PEANUTS,...,No Discount,Yes,No,Yes,No,No,No,No,16441.58141,15296.65411


In [154]:
peanuts.columns

Index(['food_game', 'UserID', 'UseCount', 'revenue', 'game_week',
       'special_discount', 'special_item', 'FAMILYGROUPNAME', 'Master_Item',
       'MENUITEMNAME', 'PRICES', 'actual_discount', 'actual_price',
       'Discount Type', 'Discount Percentage', 'first_week_discount',
       'Discount_HotDog', 'Discount_SouvCup', 'Discount_BtlWater',
       'Discount_Peanuts', 'Discount_Nachos', 'Discount_Pretzel',
       'Discount_Popcorn', 'sth_rev_game', 'total_product_rev_nonSTH'],
      dtype='object')

In [155]:
# Weight prices according to their demand.
# Prices for CL are siginificantly lower than GA / STH
# However the number of CL is also significantly lower than GA / STH
# We want to weight each actual price by the demand of item at that price point
# This negates the class imbalance issues

weights = peanuts.groupby(by = ['game_week', 'Discount Type'])['UseCount', 'actual_price'].sum(numeric_only=True).reset_index()
weights['weighted_sums'] = weights['UseCount'] * weights['actual_price']
weights['uc2'] = weights['UseCount'] ** 2
weights = weights.groupby(by = ['game_week'])['weighted_sums', 'uc2'].sum().reset_index()
weights['weighted_actual_price'] = weights['weighted_sums'] / weights['uc2']
demand = peanuts.groupby(by = ['game_week'])['UseCount'].sum(numeric_only=True).reset_index()
peanut_demand_price = pd.merge(left = weights, right = demand, on = 'game_week')
peanut_demand_price.drop(labels=['weighted_sums', 'uc2'], axis = 1, inplace=True)

/tmp/ipykernel_415796/384265912.py:7: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  weights = peanuts.groupby(by = ['game_week', 'Discount Type'])['UseCount', 'actual_price'].sum(numeric_only=True).reset_index()
/tmp/ipykernel_415796/384265912.py:10: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  weights = weights.groupby(by = ['game_week'])['weighted_sums', 'uc2'].sum().reset_index()


In [156]:
peanut_demand_price_controlled = pd.merge(left=peanut_demand_price, right=peanuts.loc[:, ['game_week', 'Discount_HotDog', 'Discount_SouvCup', 'Discount_BtlWater',
        'Discount_Nachos', 'Discount_Pretzel',
       'Discount_Popcorn']], on = 'game_week', how = 'left').drop_duplicates('game_week')

In [157]:
peanut_demand_price_controlled

,game_week,weighted_actual_price,UseCount,Discount_HotDog,Discount_SouvCup,Discount_BtlWater,Discount_Nachos,Discount_Pretzel,Discount_Popcorn
0,Game 1,4.639045,105,Yes,No,Yes,No,No,No
105,Game 2,2.420455,176,No,No,No,Yes,No,No
267,Game 3,4.640434,94,No,Yes,No,No,Yes,No
361,Game 4,4.649035,105,Yes,No,Yes,No,No,No
466,Game 5,4.646859,66,No,Yes,No,No,No,Yes
532,Game 6,4.630899,73,No,No,Yes,No,Yes,No
605,Game 7,4.651225,41,Yes,Yes,No,No,No,No
646,Game 8,4.553264,58,Yes,Yes,Yes,No,No,Yes


In [158]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df_peanut = pd.get_dummies(peanut_demand_price_controlled, columns=['game_week'])

df_peanut.loc[:, ['Discount_HotDog', 'Discount_SouvCup', 'Discount_BtlWater',
        'Discount_Nachos', 'Discount_Pretzel',
       'Discount_Popcorn']] = df_peanut.loc[:, ['Discount_HotDog', 'Discount_SouvCup', 'Discount_BtlWater',
        'Discount_Nachos', 'Discount_Pretzel',
       'Discount_Popcorn']].apply(le.fit_transform)

df_peanut['weighted_actual_price'] = np.log(df_peanut['weighted_actual_price'])
df_peanut['UseCount'] = np.log(df_peanut['UseCount'])

/tmp/ipykernel_415796/685113182.py:6: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_peanut.loc[:, ['Discount_HotDog', 'Discount_SouvCup', 'Discount_BtlWater',


In [159]:
df_peanut

,weighted_actual_price,UseCount,Discount_HotDog,Discount_SouvCup,Discount_BtlWater,Discount_Nachos,Discount_Pretzel,Discount_Popcorn,game_week_Game 1,game_week_Game 2,game_week_Game 3,game_week_Game 4,game_week_Game 5,game_week_Game 6,game_week_Game 7,game_week_Game 8
0,1.534509,4.653960,1,0,1,0,0,0,1,0,0,0,0,0,0,0
105,0.883955,5.170484,0,0,0,1,0,0,0,1,0,0,0,0,0,0
267,1.534808,4.543295,0,1,0,0,1,0,0,0,1,0,0,0,0,0
361,1.536660,4.653960,1,0,1,0,0,0,0,0,0,1,0,0,0,0
466,1.536191,4.189655,0,1,0,0,0,1,0,0,0,0,1,0,0,0
532,1.532751,4.290459,0,0,1,0,1,0,0,0,0,0,0,1,0,0
605,1.537131,3.713572,1,1,0,0,0,0,0,0,0,0,0,0,1,0
646,1.515844,4.060443,1,1,1,0,0,1,0,0,0,0,0,0,0,1


### Modeling

In [160]:
import statsmodels.api as sm

X = df_peanut.drop(labels='UseCount', axis = 1)
X = sm.add_constant(X)
y = df_peanut['UseCount']

m_peanut = sm.OLS(y, X).fit()
print('Price elasticity for peanuts is', m_peanut.params[1])

Price elasticity for peanuts is 1.6345379776174813


### BAVARIAN PRETZEL

In [161]:
bav_pret = dat.loc[dat['MENUITEMNAME'] == 'BAVARIAN PRETZEL', :]
weights = bav_pret.groupby(by = ['game_week', 'Discount Type'])['UseCount', 'actual_price'].sum(numeric_only=True).reset_index()
weights['weighted_sums'] = weights['UseCount'] * weights['actual_price']
weights['uc2'] = weights['UseCount'] ** 2
weights = weights.groupby(by = ['game_week'])['weighted_sums', 'uc2'].sum().reset_index()
weights['weighted_actual_price'] = weights['weighted_sums'] / weights['uc2']

demand = bav_pret.groupby(by = ['game_week'])['UseCount'].sum(numeric_only=True).reset_index()
bav_demand_price = pd.merge(left = weights, right = demand, on = 'game_week')
bav_demand_price.drop(labels=['weighted_sums', 'uc2'], axis = 1, inplace=True)

/tmp/ipykernel_415796/2345065388.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  weights = bav_pret.groupby(by = ['game_week', 'Discount Type'])['UseCount', 'actual_price'].sum(numeric_only=True).reset_index()
/tmp/ipykernel_415796/2345065388.py:5: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  weights = weights.groupby(by = ['game_week'])['weighted_sums', 'uc2'].sum().reset_index()


In [162]:
bav_demand_price_controlled = pd.merge(left=bav_demand_price, right=bav_pret.loc[:, ['game_week', 'Discount_HotDog', 'Discount_SouvCup', 'Discount_BtlWater',
       'Discount_Peanuts', 'Discount_Nachos', 
       'Discount_Popcorn']], on = 'game_week', how = 'left').drop_duplicates('game_week')

le = LabelEncoder()
df_bav = pd.get_dummies(bav_demand_price_controlled, columns=['game_week'])

df_bav.loc[:, ['Discount_HotDog', 'Discount_SouvCup', 'Discount_BtlWater',
       'Discount_Peanuts', 'Discount_Nachos', 
       'Discount_Popcorn']] = df_bav.loc[:, ['Discount_HotDog', 'Discount_SouvCup', 'Discount_BtlWater',
       'Discount_Peanuts', 'Discount_Nachos', 
       'Discount_Popcorn']].apply(le.fit_transform)

df_bav['weighted_actual_price'] = np.log(df_bav['weighted_actual_price'])
df_bav['UseCount'] = np.log(df_bav['UseCount'])

/tmp/ipykernel_415796/3019938838.py:8: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_bav.loc[:, ['Discount_HotDog', 'Discount_SouvCup', 'Discount_BtlWater',


In [163]:
X = df_bav.drop(labels='UseCount', axis = 1)
X = sm.add_constant(X)
y = df_bav['UseCount']

m_bav = sm.OLS(y, X).fit()
print('Price elasticity for bavarian pretzels is', m_bav.params[1])

Price elasticity for bavarian pretzels is 1.4764240307441954


### Nachos

In [164]:
nacho = dat.loc[dat['MENUITEMNAME'] == 'NACHOS', :]
weights = nacho.groupby(by = ['game_week', 'Discount Type'])['UseCount', 'actual_price'].sum(numeric_only=True).reset_index()
weights['weighted_sums'] = weights['UseCount'] * weights['actual_price']
weights['uc2'] = weights['UseCount'] ** 2
weights = weights.groupby(by = ['game_week'])['weighted_sums', 'uc2'].sum().reset_index()
weights['weighted_actual_price'] = weights['weighted_sums'] / weights['uc2']

demand = nacho.groupby(by = ['game_week'])['UseCount'].sum(numeric_only=True).reset_index()
nacho_demand_price = pd.merge(left = weights, right = demand, on = 'game_week')
nacho_demand_price.drop(labels=['weighted_sums', 'uc2'], axis = 1, inplace=True)

/tmp/ipykernel_415796/3669065834.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  weights = nacho.groupby(by = ['game_week', 'Discount Type'])['UseCount', 'actual_price'].sum(numeric_only=True).reset_index()
/tmp/ipykernel_415796/3669065834.py:5: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  weights = weights.groupby(by = ['game_week'])['weighted_sums', 'uc2'].sum().reset_index()


In [165]:
nacho_demand_price_controlled = pd.merge(left=nacho_demand_price, right=nacho.loc[:, ['game_week', 'Discount_HotDog', 'Discount_SouvCup', 'Discount_BtlWater',
       'Discount_Peanuts',  'Discount_Pretzel',
       'Discount_Popcorn']], on = 'game_week', how = 'left').drop_duplicates('game_week')

le = LabelEncoder()
df_nacho = pd.get_dummies(nacho_demand_price_controlled, columns=['game_week'])

df_nacho.loc[:, ['Discount_HotDog', 'Discount_SouvCup', 'Discount_BtlWater',
       'Discount_Peanuts', 'Discount_Pretzel',
       'Discount_Popcorn']] = df_nacho.loc[:, ['Discount_HotDog', 'Discount_SouvCup', 'Discount_BtlWater',
       'Discount_Peanuts', 'Discount_Pretzel',
       'Discount_Popcorn']].apply(le.fit_transform)

df_nacho['weighted_actual_price'] = np.log(df_nacho['weighted_actual_price'])
df_nacho['UseCount'] = np.log(df_nacho['UseCount'])

/tmp/ipykernel_415796/1135069080.py:8: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_nacho.loc[:, ['Discount_HotDog', 'Discount_SouvCup', 'Discount_BtlWater',


In [166]:
X = df_nacho.drop(labels='UseCount', axis = 1)
X = sm.add_constant(X)
y = df_nacho['UseCount']

m_nacho = sm.OLS(y, X).fit()
print('Price elasticity for nachos is', m_nacho.params[1])

Price elasticity for nachos is 1.296427876569496


### Souv Pop

In [167]:
souv_pop = dat.loc[dat['MENUITEMNAME'] == 'SOUV POPCORN', :]
weights = souv_pop.groupby(by = ['game_week', 'Discount Type'])['UseCount', 'actual_price'].sum(numeric_only=True).reset_index()
weights['weighted_sums'] = weights['UseCount'] * weights['actual_price']
weights['uc2'] = weights['UseCount'] ** 2
weights = weights.groupby(by = ['game_week'])['weighted_sums', 'uc2'].sum().reset_index()
weights['weighted_actual_price'] = weights['weighted_sums'] / weights['uc2']

demand = souv_pop.groupby(by = ['game_week'])['UseCount'].sum(numeric_only=True).reset_index()
souv_pop_demand_price = pd.merge(left = weights, right = demand, on = 'game_week')
souv_pop_demand_price.drop(labels=['weighted_sums', 'uc2'], axis = 1, inplace=True)

/tmp/ipykernel_415796/1430224477.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  weights = souv_pop.groupby(by = ['game_week', 'Discount Type'])['UseCount', 'actual_price'].sum(numeric_only=True).reset_index()
/tmp/ipykernel_415796/1430224477.py:5: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  weights = weights.groupby(by = ['game_week'])['weighted_sums', 'uc2'].sum().reset_index()


In [168]:
souv_pop_demand_price_controlled = pd.merge(left=souv_pop_demand_price, right=souv_pop.loc[:, ['game_week', 'Discount_HotDog', 'Discount_SouvCup', 'Discount_BtlWater',
       'Discount_Peanuts', 'Discount_Nachos', 'Discount_Pretzel']], on = 'game_week', how = 'left').drop_duplicates('game_week')

le = LabelEncoder()
df_souv_pop = pd.get_dummies(souv_pop_demand_price_controlled, columns=['game_week'])

df_souv_pop.loc[:, ['Discount_HotDog', 'Discount_SouvCup', 'Discount_BtlWater',
       'Discount_Peanuts', 'Discount_Nachos', 'Discount_Pretzel']] = df_souv_pop.loc[:, ['Discount_HotDog', 'Discount_SouvCup', 'Discount_BtlWater',
       'Discount_Peanuts', 'Discount_Nachos', 'Discount_Pretzel']].apply(le.fit_transform)

df_souv_pop['weighted_actual_price'] = np.log(df_souv_pop['weighted_actual_price'])
df_souv_pop['UseCount'] = np.log(df_souv_pop['UseCount'])

/tmp/ipykernel_415796/394850179.py:7: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_souv_pop.loc[:, ['Discount_HotDog', 'Discount_SouvCup', 'Discount_BtlWater',


In [169]:
X = df_souv_pop.drop(labels='UseCount', axis = 1)
X = sm.add_constant(X)
y = df_souv_pop['UseCount']

m_souv_pop = sm.OLS(y, X).fit()
print('Price elasticity for souvenir popcorn is', m_souv_pop.params[1])

Price elasticity for souvenir popcorn is 0.8161023513398378


### Hot Dog

In [170]:
hotdog = dat.loc[dat['MENUITEMNAME'] == 'HOT DOG', :]
weights = hotdog.groupby(by = ['game_week', 'Discount Type'])['UseCount', 'actual_price'].sum(numeric_only=True).reset_index()
weights['weighted_sums'] = weights['UseCount'] * weights['actual_price']
weights['uc2'] = weights['UseCount'] ** 2
weights = weights.groupby(by = ['game_week'])['weighted_sums', 'uc2'].sum().reset_index()
weights['weighted_actual_price'] = weights['weighted_sums'] / weights['uc2']

demand = hotdog.groupby(by = ['game_week'])['UseCount'].sum(numeric_only=True).reset_index()
hotdog_demand_price = pd.merge(left = weights, right = demand, on = 'game_week')
hotdog_demand_price.drop(labels=['weighted_sums', 'uc2'], axis = 1, inplace=True)

/tmp/ipykernel_415796/3883914336.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  weights = hotdog.groupby(by = ['game_week', 'Discount Type'])['UseCount', 'actual_price'].sum(numeric_only=True).reset_index()
/tmp/ipykernel_415796/3883914336.py:5: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  weights = weights.groupby(by = ['game_week'])['weighted_sums', 'uc2'].sum().reset_index()


In [171]:
hotdog_demand_price_controlled = pd.merge(left=hotdog_demand_price, right=hotdog.loc[:, ['game_week',  'Discount_SouvCup', 'Discount_BtlWater',
       'Discount_Peanuts', 'Discount_Nachos', 'Discount_Pretzel',
       'Discount_Popcorn']], on = 'game_week', how = 'left').drop_duplicates('game_week')

le = LabelEncoder()
df_hotdog = pd.get_dummies(hotdog_demand_price_controlled, columns=['game_week'])

df_hotdog.loc[:, [ 'Discount_SouvCup', 'Discount_BtlWater',
       'Discount_Peanuts', 'Discount_Nachos', 'Discount_Pretzel',
       'Discount_Popcorn']] = df_hotdog.loc[:, ['Discount_SouvCup', 'Discount_BtlWater',
       'Discount_Peanuts', 'Discount_Nachos', 'Discount_Pretzel',
       'Discount_Popcorn']].apply(le.fit_transform)

df_hotdog['weighted_actual_price'] = np.log(df_hotdog['weighted_actual_price'])
df_hotdog['UseCount'] = np.log(df_hotdog['UseCount'])

/tmp/ipykernel_415796/1114081131.py:8: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_hotdog.loc[:, [ 'Discount_SouvCup', 'Discount_BtlWater',


In [172]:
X = df_hotdog.drop(labels='UseCount', axis = 1)
X = sm.add_constant(X)
y = df_hotdog['UseCount']

m_hotdog = sm.OLS(y, X).fit()
print('Price elasticity for hot dog is', m_hotdog.params[1])

Price elasticity for hot dog is 0.8348131121438419


### Bottled Water (non 1L)

In [173]:
btlwater = dat.loc[dat['MENUITEMNAME'] == 'BTL DEJA BLUE', :]
weights = btlwater.groupby(by = ['game_week', 'Discount Type'])['UseCount', 'actual_price'].sum(numeric_only=True).reset_index()
weights['weighted_sums'] = weights['UseCount'] * weights['actual_price']
weights['uc2'] = weights['UseCount'] ** 2
weights = weights.groupby(by = ['game_week'])['weighted_sums', 'uc2'].sum().reset_index()
weights['weighted_actual_price'] = weights['weighted_sums'] / weights['uc2']

demand = btlwater.groupby(by = ['game_week'])['UseCount'].sum(numeric_only=True).reset_index()
btlwater_demand_price = pd.merge(left = weights, right = demand, on = 'game_week')
btlwater_demand_price.drop(labels=['weighted_sums', 'uc2'], axis = 1, inplace=True)

/tmp/ipykernel_415796/3487976040.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  weights = btlwater.groupby(by = ['game_week', 'Discount Type'])['UseCount', 'actual_price'].sum(numeric_only=True).reset_index()
/tmp/ipykernel_415796/3487976040.py:5: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  weights = weights.groupby(by = ['game_week'])['weighted_sums', 'uc2'].sum().reset_index()


In [174]:
btlwater_demand_price_controlled = pd.merge(left=btlwater_demand_price, right=btlwater.loc[:, ['game_week', 'Discount_HotDog', 'Discount_SouvCup', 
       'Discount_Peanuts', 'Discount_Nachos', 'Discount_Pretzel',
       'Discount_Popcorn']], on = 'game_week', how = 'left').drop_duplicates('game_week')

le = LabelEncoder()
df_btlwater = pd.get_dummies(btlwater_demand_price_controlled, columns=['game_week'])

df_btlwater.loc[:, ['Discount_HotDog', 'Discount_SouvCup', 
       'Discount_Peanuts', 'Discount_Nachos', 'Discount_Pretzel',
       'Discount_Popcorn']] = df_btlwater.loc[:, ['Discount_HotDog', 'Discount_SouvCup', 
       'Discount_Peanuts', 'Discount_Nachos', 'Discount_Pretzel',
       'Discount_Popcorn']].apply(le.fit_transform)

df_btlwater['weighted_actual_price'] = np.log(df_btlwater['weighted_actual_price'])
df_btlwater['UseCount'] = np.log(df_btlwater['UseCount'])

/tmp/ipykernel_415796/1160784298.py:8: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_btlwater.loc[:, ['Discount_HotDog', 'Discount_SouvCup',


In [175]:
X = df_btlwater.drop(labels='UseCount', axis = 1)
X = sm.add_constant(X)
y = df_btlwater['UseCount']

m_btlwater = sm.OLS(y, X).fit()
print('Price elasticity for bottled water is', m_btlwater.params[1])

Price elasticity for bottled water is 1.4582735241204785


### Souvenir Soda (32 oz)

In [176]:
souv_soda = dat.loc[dat['MENUITEMNAME'] == 'SOUV CUP 32', :]
weights = souv_soda.groupby(by = ['game_week', 'Discount Type'])['UseCount', 'actual_price'].sum(numeric_only=True).reset_index()
weights['weighted_sums'] = weights['UseCount'] * weights['actual_price']
weights['uc2'] = weights['UseCount'] ** 2
weights = weights.groupby(by = ['game_week'])['weighted_sums', 'uc2'].sum().reset_index()
weights['weighted_actual_price'] = weights['weighted_sums'] / weights['uc2']

demand = souv_soda.groupby(by = ['game_week'])['UseCount'].sum(numeric_only=True).reset_index()
souv_soda_demand_price = pd.merge(left = weights, right = demand, on = 'game_week')
souv_soda_demand_price.drop(labels=['weighted_sums', 'uc2'], axis = 1, inplace=True)

/tmp/ipykernel_415796/3570251779.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  weights = souv_soda.groupby(by = ['game_week', 'Discount Type'])['UseCount', 'actual_price'].sum(numeric_only=True).reset_index()
/tmp/ipykernel_415796/3570251779.py:5: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  weights = weights.groupby(by = ['game_week'])['weighted_sums', 'uc2'].sum().reset_index()


In [177]:
souv_soda_demand_price_controlled = pd.merge(left=souv_soda_demand_price, right=souv_soda.loc[:, ['game_week', 'Discount_HotDog',  'Discount_BtlWater',
       'Discount_Peanuts', 'Discount_Nachos', 'Discount_Pretzel',
       'Discount_Popcorn']], on = 'game_week', how = 'left').drop_duplicates('game_week')

le = LabelEncoder()
df_souv_soda = pd.get_dummies(souv_soda_demand_price_controlled, columns=['game_week'])

df_souv_soda.loc[:, ['Discount_HotDog',  'Discount_BtlWater',
       'Discount_Peanuts', 'Discount_Nachos', 'Discount_Pretzel',
       'Discount_Popcorn']] = df_souv_soda.loc[:, ['Discount_HotDog', 'Discount_BtlWater',
       'Discount_Peanuts', 'Discount_Nachos', 'Discount_Pretzel',
       'Discount_Popcorn']].apply(le.fit_transform)

df_souv_soda['weighted_actual_price'] = np.log(df_souv_soda['weighted_actual_price'])
df_souv_soda['UseCount'] = np.log(df_souv_soda['UseCount'])

/tmp/ipykernel_415796/3437261013.py:8: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_souv_soda.loc[:, ['Discount_HotDog',  'Discount_BtlWater',


In [178]:
X = df_souv_soda.drop(labels='UseCount', axis = 1)
X = sm.add_constant(X)
y = df_souv_soda['UseCount']

m_souv_soda = sm.OLS(y, X).fit()
print('Price elasticity for bottled water is', m_souv_soda.params[1])

Price elasticity for bottled water is 1.7736187044042304


# Question 2

### Nachos

In [194]:
m_nacho.params[1:8]

weighted_actual_price    1.296428
Discount_HotDog         -0.144358
Discount_SouvCup        -0.230704
Discount_BtlWater        0.346280
Discount_Peanuts         1.375867
Discount_Pretzel         0.100675
Discount_Popcorn         0.159259
dtype: float64

As seen above:

- If there is a discount for hotdog, the price elasticity decreases by 0.14
- If there is a discount for souvenir soda 32oz, the price elasticity decreases by 0.23 
- If there is a discount for bottle water, the price elasticity increases by 0.35
- If there is a discount for peanut, the price elasticity increases by 1.38
- If there is a discount for pretzel, the price elasticity increases by 0.1
- If there is a discount for souvenir popcorn, the price elasticity increases by 0.16

### Souvenir Popcorn

In [195]:
m_souv_pop.params[1:8]

weighted_actual_price    0.816102
Discount_HotDog         -0.255825
Discount_SouvCup         1.138184
Discount_BtlWater        0.579297
Discount_Peanuts        -0.097276
Discount_Nachos         -0.097276
Discount_Pretzel        -0.381371
dtype: float64

As seen above:

- If there is a discount for hotdog, the price elasticity decreases by 0.26
- If there is a discount for souvenir soda 32oz, the price elasticity increases by 1.14
- If there is a discount for bottle water, the price elasticity increases by 0.58
- If there is a discount for peanut, the price elasticity decreases by 0.1
- If there is a discount for nachos, the price elasticity decreases by 0.1
- If there is a discount for pretzel, the price elasticity decreases by 0.38

### Hot Dog

In [196]:
m_hotdog.params[1:8]

weighted_actual_price    0.834813
Discount_SouvCup         1.213797
Discount_BtlWater        1.949597
Discount_Peanuts         0.095981
Discount_Nachos          0.095981
Discount_Pretzel        -0.687455
Discount_Popcorn        -0.247192
dtype: float64

- If there is a discount for souvenir soda 32oz, the price elasticity increases by 1.21
- If there is a discount for bottle water, the price elasticity increases by 1.95
- If there is a discount for peanut, the price elasticity increases by 0.1
- If there is a discount for nachos, the price elasticity increases by 0.1
- If there is a discount for pretzel, the price elasticity decreases by 0.68
- If there is a discount for souvenir popcorn, the price elasticity decreases by 0.24

### Peanuts

In [197]:
m_peanut.params[1:8]

weighted_actual_price    1.634538
Discount_HotDog          0.057168
Discount_SouvCup        -0.024896
Discount_BtlWater        0.246437
Discount_Nachos          1.098729
Discount_Pretzel         0.180612
Discount_Popcorn        -0.016240
dtype: float64

As seen above:

- If there is a discount for hotdog, the price elasticity increases by 0.06
- If there is a discount for souvenir soda 32oz, the price elasticity decreases by 0.02
- If there is a discount for bottle water, the price elasticity increases by 0.25
- If there is a discount for nachos, the price elasticity increases by 1.1
- If there is a discount for pretzel, the price elasticity increases by 0.18
- If there is a discount for souvenir popcorn, the price elasticity decreases by 0.02

### Pretzel

In [198]:
m_bav.params[1:8]

weighted_actual_price    1.476424
Discount_HotDog         -0.841341
Discount_SouvCup         0.921666
Discount_BtlWater        1.114007
Discount_Peanuts        -0.033668
Discount_Nachos         -0.033668
Discount_Popcorn        -0.668894
dtype: float64

As seen above:

- If there is a discount for hotdog, the price elasticity decreases by 0.84
- If there is a discount for souvenir soda 32oz, the price elasticity increases by 0.92
- If there is a discount for bottle water, the price elasticity increases by 1.11
- If there is a discount for peanuts, the price elasticity decreases by 0.03
- If there is a discount for nachos, the price elasticity decreases by 0.03
- If there is a discount for souvenir popcorn, the price elasticity decreases by 0.67

### Bottled Water

In [199]:
m_btlwater.params[1:8]

weighted_actual_price    1.458274
Discount_HotDog          1.604887
Discount_SouvCup        -0.639235
Discount_Peanuts         0.098833
Discount_Nachos          0.098833
Discount_Pretzel         1.213075
Discount_Popcorn         0.591864
dtype: float64

As seen above:

- If there is a discount for hotdog, the price elasticity increases by 1.6
- If there is a discount for souvenir soda 32oz, the price elasticity decreases by 0.64
- If there is a discount for peanuts, the price elasticity increases by 0.1
- If there is a discount for nachos, the price elasticity increases by 0.1
- If there is a discount for pretzel, the price elasticity increases by 1.21
- If there is a discount for souvenir popcorn, the price elasticity increases by 0.59

### Souvenir Soda

In [200]:
m_souv_soda.params[1:8]

weighted_actual_price    1.773619
Discount_HotDog          0.821506
Discount_BtlWater       -0.494552
Discount_Peanuts        -0.067103
Discount_Nachos         -0.067103
Discount_Pretzel         0.975265
Discount_Popcorn         1.209335
dtype: float64

As seen above:

- If there is a discount for hotdog, the price elasticity increases by 0.82
- If there is a discount for bottle water, the price elasticity decreases by 0.49
- If there is a discount for peanuts, the price elasticity decreases by 0.07
- If there is a discount for nachos, the price elasticity decreases by 0.07
- If there is a discount for pretzel, the price elasticity increases by 0.98
- If there is a discount for souvenir popcorn, the price elasticity increases by 1.21